In [2]:
import pandas as pd
import numpy as np
from untrade.client import Client
from scipy.signal import argrelextrema

In [7]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    i = 26
    while i < len(df) - 6:
        # Finding both peaks (highs) and troughs (lows) in the last 26 candles
        extremas = sorted(
            list(argrelextrema(df['close'].values[i-26:i], np.greater)[0]) +
            list(argrelextrema(df['close'].values[i-26:i], np.less)[0])
        )
        
        # Ensure at least two points exist
        if len(extremas) < 2:
            i += 1
            continue
        
        # Identifying the last two extrema
        point_A = extremas[-2]
        point_B = extremas[-1]
        point_C = i  # Append current index
        
        price_A = df['close'].iloc[i-26+point_A]
        price_B = df['close'].iloc[i-26+point_B]
        price_C = df['close'].iloc[i]
        
        # Determine trend direction
        if price_B > price_A:  # Bullish setup (long trade)
            retracement_level = price_B - 0.618 * (price_B - price_A)
            
            if abs(price_C - retracement_level) <= 0.01 * price_B:
                lower_bound = retracement_level * 0.97
                upper_bound = retracement_level * 1.03
                if df['close'].iloc[i+1:i+6].between(lower_bound, upper_bound).all():
                    delta = price_B - price_A
                    df.loc[df.index[i], 'signals'] = 1
                    df.loc[df.index[i], 'SL'] = (price_B - 0.78 * delta) * 0.97  # Apply buffer
                    df.loc[df.index[i], 'TP'] = (price_B + 1.2 * delta) * 1.03  # Apply buffer
                    df.loc[df.index[i], 'trade'] = 1
                    df.loc[df.index[i], 'trade_type'] = 'long'
        
        elif price_B < price_A:  # Bearish setup (short trade)
            retracement_level = price_B + 0.618 * (price_A - price_B)
            
            if abs(price_C - retracement_level) <= 0.01 * price_B:
                lower_bound = retracement_level * 0.97
                upper_bound = retracement_level * 1.03
                if df['close'].iloc[i+1:i+6].between(lower_bound, upper_bound).all():
                    delta = price_A - price_B
                    df.loc[df.index[i], 'signals'] = -1
                    df.loc[df.index[i], 'SL'] = (price_B + 0.78 * delta) * 1.03  # Apply buffer
                    df.loc[df.index[i], 'TP'] = (price_B - 1.2 * delta) * 0.97  # Apply buffer
                    df.loc[df.index[i], 'trade'] = -1
                    df.loc[df.index[i], 'trade_type'] = 'short'
        
        i += 1
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP']]
    
    return df


def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type']]
    
    data["leverage"] = 5

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_96290/42124440.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 703,
      "Leverage Applied": 5.0,
      "Winning Trades": 317,
      "Losing Trades": 386,
      "No. of Long Trades": 284,
      "No. of Short Trades": 419,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 45.092461,
      "Winning Streak": 8,
      "Losing Streak": 9,
      "Gross Profit": 1004.212784,
      "Net Profit": -4268.287216,
      "Average Profit": -6.071532,
      "Maximum Drawdown(%)": 152.053113,
      "Average Drawdown(%)": 103.375755,
      "Largest Win": 2155.013946,
      "Average Win": 108.731065,
      "Largest Loss": -4176.181901,
      "Average Loss": -100.352422,
      "Maximum Holding Time": "29 days 12:0:0",
      "Average Holding Time": "1 days 6:4:16",
      "Maximum Adverse Excursion": 101.